In [1]:
import heapq
import warnings

import mglearn as mglearn
import np as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import sklearn_evaluation
from sklearn.calibration import CalibratedClassifierCV
from sklearn.exceptions import ConvergenceWarning
from sklearn.feature_selection import SelectFromModel, SelectPercentile, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.svm import SVC, LinearSVC

!pip install numpy scipy scikit-learn matplotlib pandas
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.ensemble import StackingClassifier
import heapq


You should consider upgrading via the '/Users/engineer/workspace/cse590-machine-learning/venv/bin/python -m pip install --upgrade pip' command.


read data from files

In [24]:
X = pd.read_csv("../dataset/extracted_features.csv").values
y = pd.read_csv("../dataset/labels.csv").values
# images = pd.read_csv("../dataset/raw_images.csv").values
X_train_default, X_test_default, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
)
y_train = y_train.ravel()


perform 4 feature selection methods

In [25]:
index_for_removal = np.array(
    [
        0, 10, 14, 17, 25, 26, 35, 39, 49, 59, 62,
    ]
)
X_train_zero_variance = np.delete(X_train_default, index_for_removal, 1)
X_test_zero_variance = np.delete(X_test_default, index_for_removal, 1)
#
# select = SelectPercentile(percentile=90)
# select.fit(X_train_zero_variance, y_train)
# X_train_univariate = select.transform(X_train_zero_variance)
# X_test_univariate = select.transform(X_test_zero_variance)
#
# rf = RandomForestClassifier(
#     n_estimators=200,
#     max_features=3,
# ).fit(X_train_default, y_train)
#
# select_sfm = SelectFromModel(
#     rf,
#     threshold='0.1*mean',
# )
# select_sfm.fit(X_train_univariate, y_train)
# X_train_from_model = select_sfm.transform(X_train_univariate)
# X_test_from_model = select_sfm.transform(X_test_univariate)
#
# select_rfe = RFE(
#     RandomForestClassifier(
#         n_estimators=200, random_state=42
#     ),
#     n_features_to_select=40
# )
#
# select_rfe.fit(X_train_from_model, y_train)
# X_train_zero_variance = select_rfe.transform(X_train_from_model)
# X_test_zero_variance = select_rfe.transform(X_test_from_model)


standard scaler: default and post-feature selection

In [26]:
scaler_standard_default = StandardScaler().fit(X_train_default)
X_train_standard_default = scaler_standard_default.transform(X_train_default)
X_test_standard_default = scaler_standard_default.transform(X_test_default)

scaler_standard_refined = StandardScaler().fit(X_train_zero_variance)
X_train_standard_refined = scaler_standard_refined.transform(X_train_zero_variance)
X_test_standard_refined = scaler_standard_refined.transform(X_test_zero_variance)


robust scaler: default and post-feature selection

In [27]:
scaler_robust_default = RobustScaler().fit(X_train_default)
X_train_robust_default = scaler_robust_default.transform(X_train_default)
X_test_robust_default = scaler_robust_default.transform(X_test_default)

scaler_robust_refined = RobustScaler().fit(X_train_zero_variance)
X_train_robust_refined = scaler_robust_refined.transform(X_train_zero_variance)
X_test_robust_refined = scaler_robust_refined.transform(X_test_zero_variance)


minmax scaler: default and post-feature selection

In [28]:
scaler_minmax_default = MinMaxScaler().fit(X_train_default)
X_train_minmax_default = scaler_minmax_default.transform(X_train_default)
X_test_minmax_default = scaler_minmax_default.transform(X_test_default)

scaler_minmax_refined = MinMaxScaler().fit(X_train_zero_variance)
X_train_minmax_refined = scaler_minmax_refined.transform(X_train_zero_variance)
X_test_minmax_refined = scaler_minmax_refined.transform(X_test_zero_variance)


PCA: default and post-feature selection

In [29]:
pca_default = PCA(n_components=56, random_state=42)
X_train_pca_default = pca_default.fit_transform(X_train_standard_default)
X_test_pca_default = pca_default.transform(X_test_standard_default)

pca_refined = PCA(n_components=37, random_state=42)
X_train_pca_refined = pca_refined.fit_transform(X_train_standard_refined)
X_test_pca_refined = pca_refined.transform(X_test_standard_refined)


In [30]:
data_train_default = {
    'default': X_train_default,
    'standard': X_train_standard_default,
    'robust': X_train_robust_default,
    'minmax': X_train_minmax_default,
    'pca': X_train_pca_default
}
data_test_default = {
    'default': X_test_default,
    'standard': X_test_standard_default,
    'robust': X_test_robust_default,
    'minmax': X_test_minmax_default,
    'pca': X_test_pca_default
}
data_default = {
    'train': data_train_default,
    'test': data_test_default
}


In [31]:
data_train_refined = {
    'default': X_train_zero_variance,
    'standard': X_train_standard_refined,
    'robust': X_train_robust_refined,
    'minmax': X_train_minmax_refined,
    'pca': X_train_pca_refined
}
data_test_refined = {
    'default': X_test_zero_variance,
    'standard': X_test_standard_refined,
    'robust': X_test_robust_refined,
    'minmax': X_test_minmax_refined,
    'pca': X_test_pca_refined
}
data_refined = {
    'train': data_train_refined,
    'test': data_test_refined
}


In [32]:
score_types = {'accuracy', 'auc', 'f1'}


def canTrain(key, canAcceptNegative):
    if key == 'standard' or key == 'robust' or key == 'pca':
        return canAcceptNegative
    else:
        return True


def grid_search_all_data(classifier, param_grid, data, canAcceptNegative):
    grid_results = {}

    for key in data['train']:
        if not canTrain(key, canAcceptNegative):
            continue

        X_train_data = data['train'][key]
        grid_search = GridSearchCV(
            classifier,
            param_grid,
            cv=4,
            return_train_score=True
        )
        grid_results[key] = grid_search.fit(X_train_data, y_train)

    # print_grid_search_results(grid_search, hyper_parameters)
    return grid_results


In [33]:



def score_model_after_cv(classifier, grid_results, data, requiresCalibration, canAcceptNegative):
    scores = {}

    for key in grid_results:
        if not canTrain(key, canAcceptNegative):
            continue

        clf = classifier.set_params(**grid_results[key].best_params_)
        if requiresCalibration:
            clf = CalibratedClassifierCV(clf)
        X_train_data = data['train'][key]
        X_test_data = data['test'][key]
        clf.fit(X_train_data, y_train)
        scores[key] = {}
        scores[key]['train acc'] = clf.score(X_train_data, y_train)
        scores[key]['test acc'] = clf.score(X_test_data, y_test)
        y_pred = clf.predict_proba(X_test_data)
        scores[key]['auc'] = roc_auc_score(y_test, y_pred, multi_class='ovr')
        # scores[key]['f1'] = f1_score(y_test, y_pred)

    return scores

    # print(mlp.score(X_train_minmax, y_train))
    # print(mlp.score(X_test_minmax, y_test))
    # print(roc_auc_score(y_test, mlp.predict_proba(X_test_minmax), multi_class='ovr'))


In [69]:
def display_grid_results(grid_results):
    keys = list(grid_results.keys())
    param_by_data = {}
    param_names = list(grid_results[keys[0]].best_params_.keys())
    print(param_names)

    for key in grid_results:
        param_values = []
        for param in grid_results[key].best_params_:
            param_values.append(grid_results[key].best_params_[param])
        param_by_data[key] = param_values
    print(param_by_data)
    df = pd.DataFrame.from_dict(param_by_data,  orient='index', columns=param_names)
    return df

KNN

In [34]:
knn_param_grid = {
    'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10]
}

knn_grid_results_default = grid_search_all_data(KNeighborsClassifier(), knn_param_grid, data_default, True)

In [35]:
knn_grid_results_refined = grid_search_all_data(KNeighborsClassifier(), knn_param_grid, data_refined, True)

In [57]:
# pd.DataFrame(knn_default_grid_results)
print(knn_grid_results_default['default'].best_params_)

{'n_neighbors': 5}


In [36]:
knn_scores_default = score_model_after_cv(
    classifier=KNeighborsClassifier(), grid_results=knn_grid_results_default, data=data_default,
    requiresCalibration=False, canAcceptNegative=True
)

In [37]:
pd.DataFrame(knn_scores_default)


,default,standard,robust,minmax,pca
train acc,0.909296,0.866479,0.674930,0.898028,0.867042
test acc,0.853604,0.790541,0.581081,0.828829,0.790541
auc,0.974448,0.939579,0.883708,0.952293,0.939512


In [38]:
knn_scores_refined = score_model_after_cv(
    classifier=KNeighborsClassifier(), grid_results=knn_grid_results_refined, data=data_refined,
    requiresCalibration=False, canAcceptNegative=True
)


In [39]:
pd.DataFrame(knn_scores_refined)

,default,standard,robust,minmax,pca
train acc,0.908732,0.868169,0.674930,0.896901,0.827606
test acc,0.851351,0.788288,0.581081,0.826577,0.750000
auc,0.974289,0.939485,0.883708,0.952151,0.949434


In [73]:
display_grid_results(knn_grid_results_refined)

['n_neighbors']
{'default': [5], 'standard': [5], 'robust': [9], 'minmax': [4], 'pca': [10]}


,n_neighbors
default,5
standard,5
robust,9
minmax,4
pca,10


Logreg

In [40]:
logreg_param_grid = {
    'C': np.linspace(0, 5, 20)
}


In [72]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    logreg_grid_results_default = grid_search_all_data(LogisticRegression(), logreg_param_grid, data_default, True)

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:811: RuntimeWarning: divide by zero encountered in double_scalars
  args=(X, target, 1.0 / C, sample_weight),
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:301: RuntimeWarning: invalid value encountered in double_scalars
  loss += 0.5 * alpha * squared_norm(w)
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:352: RuntimeWarning: invalid value encountered in multiply
  grad[:, :n_features] += alpha * w
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:811: RuntimeWarning: divide by zero encountered in double_scalars
  args=(X, target, 1.0 / C, sample_weight),
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logisti

In [41]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    logreg_grid_results_refined = grid_search_all_data(LogisticRegression(), logreg_param_grid, data_refined, True)

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:811: RuntimeWarning: divide by zero encountered in double_scalars
  args=(X, target, 1.0 / C, sample_weight),
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:301: RuntimeWarning: invalid value encountered in double_scalars
  loss += 0.5 * alpha * squared_norm(w)
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:352: RuntimeWarning: invalid value encountered in multiply
  grad[:, :n_features] += alpha * w
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:811: RuntimeWarning: divide by zero encountered in double_scalars
  args=(X, target, 1.0 / C, sample_weight),
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logisti

In [81]:
logreg_scores_default = score_model_after_cv(
    classifier=LogisticRegression(), grid_results=logreg_grid_results_default, data=data_default,
    requiresCalibration=False, canAcceptNegative=True
)


/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

In [82]:
pd.DataFrame(logreg_scores_default)


,default,standard,robust,minmax,pca
train acc,0.976338,0.957746,0.960000,0.941972,0.957746
test acc,0.912162,0.916667,0.887387,0.923423,0.916667
auc,0.991583,0.992378,0.988755,0.992831,0.992378


In [42]:
logreg_scores_refined = score_model_after_cv(
    classifier=LogisticRegression(), grid_results=logreg_grid_results_refined, data=data_refined,
    requiresCalibration=False, canAcceptNegative=True
)


/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

In [48]:
pd.DataFrame(logreg_scores_refined)


,default,standard,robust,minmax,pca
train acc,0.975775,0.956620,0.958873,0.941972,0.934648
test acc,0.912162,0.918919,0.894144,0.923423,0.898649
auc,0.991622,0.992392,0.989629,0.992803,0.983542


In [74]:
display_grid_results(logreg_grid_results_refined)

['C']
{'default': [0.2631578947368421], 'standard': [0.2631578947368421], 'robust': [0.2631578947368421], 'minmax': [4.7368421052631575], 'pca': [0.2631578947368421]}


,C
default,0.263158
standard,0.263158
robust,0.263158
minmax,4.736842
pca,0.263158


# LSVC

In [44]:
lsvc_param_grid = {
    'C': np.linspace(0, 5, 20)
}


In [145]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    lsvc_grid_results_default = grid_search_all_data(LinearSVC(), lsvc_param_grid, data_default, True)


/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
4 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_classes.py", line 257, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "/Users/engineer/workspace/

In [45]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    lsvc_grid_results_refined = grid_search_all_data(LinearSVC(), lsvc_param_grid, data_refined, True)


/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
4 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_classes.py", line 257, in fit
    self.coef_, self.intercept_, self.n_iter_ = _fit_liblinear(
  File "/Users/engineer/workspace/

In [147]:
lsvc_scores_default = score_model_after_cv(
    classifier=LinearSVC(), grid_results=lsvc_grid_results_default, data=data_default, requiresCalibration=True,
    canAcceptNegative=True
)

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to co

In [148]:

pd.DataFrame(lsvc_scores_default)

,default,standard,robust,minmax,pca
train acc,0.943662,0.947606,0.949296,0.930141,0.947606
test acc,0.912162,0.916667,0.916667,0.912162,0.916667
auc,0.985756,0.986922,0.986757,0.987580,0.986886


In [46]:
lsvc_scores_refined = score_model_after_cv(
    classifier=LinearSVC(), grid_results=lsvc_grid_results_refined, data=data_refined, requiresCalibration=True,
    canAcceptNegative=True
)

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to co

In [49]:
pd.DataFrame(lsvc_scores_refined)

,default,standard,robust,minmax,pca
train acc,0.945352,0.948169,0.948169,0.938592,0.924507
test acc,0.912162,0.918919,0.916667,0.921171,0.887387
auc,0.985887,0.986966,0.986737,0.987798,0.975472


In [75]:
display_grid_results(lsvc_grid_results_refined)

['C']
{'default': [0.2631578947368421], 'standard': [0.2631578947368421], 'robust': [0.2631578947368421], 'minmax': [1.8421052631578947], 'pca': [0.5263157894736842]}


,C
default,0.263158
standard,0.263158
robust,0.263158
minmax,1.842105
pca,0.526316


## MNB

In [51]:
mnb_param_grid = {
    'alpha': np.linspace(0, 2, 20)
}

In [155]:
mnb_grid_results_default = grid_search_all_data(MultinomialNB(), mnb_param_grid, data_default, False)


/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result 

In [52]:
mnb_grid_results_refined = grid_search_all_data(MultinomialNB(), mnb_param_grid, data_refined, False)

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result 

In [140]:
mnb_scores_default = score_model_after_cv(
    classifier=MultinomialNB(),
    grid_results=mnb_grid_results_default,
    data=data_default,
    requiresCalibration=False,
    canAcceptNegative=True
)

In [157]:

pd.DataFrame(mnb_scores_default)

,default,minmax
train acc,0.901408,0.842254
test acc,0.898649,0.840090
auc,0.987748,0.978726


In [53]:

mnb_scores_refined = score_model_after_cv(
    classifier=MultinomialNB(),
    grid_results=mnb_grid_results_refined,
    data=data_refined,
    requiresCalibration=False,
    canAcceptNegative=True
)


In [54]:
pd.DataFrame(mnb_scores_refined)

,default,minmax
train acc,0.901408,0.841690
test acc,0.898649,0.840090
auc,0.987748,0.978885


In [76]:
display_grid_results(mnb_grid_results_refined)

['alpha']
{'default': [0.21052631578947367], 'minmax': [0.10526315789473684]}


,alpha
default,0.210526
minmax,0.105263


## Random Forest

In [55]:
rf_param_grid = {
    'max_features': [1, 3, 5],
    'n_estimators': [200, 400, 600],
    'random_state': [42]
}

In [162]:

rf_grid_results_default = grid_search_all_data(RandomForestClassifier(), rf_param_grid, data_default, True)

In [56]:

rf_grid_results_refined = grid_search_all_data(RandomForestClassifier(), rf_param_grid, data_refined, True)

In [164]:
rf_scores_default = score_model_after_cv(
    classifier=RandomForestClassifier(),
    grid_results=rf_grid_results_default,
    data=data_default,
    requiresCalibration=False,
    canAcceptNegative=True
)

In [165]:

pd.DataFrame(rf_scores_default)

,default,standard,robust,minmax,pca
train acc,0.991549,0.991549,0.991549,0.991549,0.991549
test acc,0.889640,0.889640,0.889640,0.889640,0.876126
auc,0.986570,0.986570,0.986570,0.986570,0.984269


In [57]:

rf_scores_refined = score_model_after_cv(
    classifier=RandomForestClassifier(),
    grid_results=rf_grid_results_refined,
    data=data_refined,
    requiresCalibration=False,
    canAcceptNegative=True
)

In [58]:

pd.DataFrame(rf_scores_refined)

,default,standard,robust,minmax,pca
train acc,0.991549,0.991549,0.991549,0.991549,0.991549
test acc,0.891892,0.891892,0.891892,0.891892,0.844595
auc,0.985364,0.985364,0.985364,0.985364,0.980257


In [77]:
display_grid_results(rf_grid_results_refined)

['max_features', 'n_estimators', 'random_state']
{'default': [3, 400, 42], 'standard': [3, 400, 42], 'robust': [3, 400, 42], 'minmax': [3, 400, 42], 'pca': [5, 600, 42]}


,max_features,n_estimators,random_state
default,3,400,42
standard,3,400,42
robust,3,400,42
minmax,3,400,42
pca,5,600,42


## GBRT


In [59]:
gbrt_param_grid = {
    'n_estimators': [200, 400],
    'max_depth': [1, 3],
    'learning_rate': [0.01, 0.1],
    'random_state': [42]
}

In [169]:

gbrt_grid_results_default = grid_search_all_data(GradientBoostingClassifier(), gbrt_param_grid, data_default, True)

In [61]:

gbrt_grid_results_refined = grid_search_all_data(GradientBoostingClassifier(), gbrt_param_grid, data_refined, True)

In [171]:

gbrt_scores_default = score_model_after_cv(
    classifier=GradientBoostingClassifier(),
    grid_results=gbrt_grid_results_default,
    data=data_default,
    requiresCalibration=False,
    canAcceptNegative=True
)

In [172]:

pd.DataFrame(gbrt_scores_default)

,default,standard,robust,minmax,pca
train acc,0.974648,0.974648,0.974648,0.974648,0.991549
test acc,0.878378,0.878378,0.878378,0.878378,0.858108
auc,0.988681,0.988681,0.988681,0.988681,0.986279


In [62]:

gbrt_scores_refined = score_model_after_cv(
    classifier=GradientBoostingClassifier(),
    grid_results=gbrt_grid_results_refined,
    data=data_refined,
    requiresCalibration=False,
    canAcceptNegative=True
)

In [63]:

pd.DataFrame(gbrt_scores_refined)

,default,standard,robust,minmax,pca
train acc,0.975211,0.975211,0.975211,0.975211,0.971268
test acc,0.887387,0.887387,0.887387,0.887387,0.835586
auc,0.988685,0.988685,0.988685,0.988685,0.976472


In [78]:
display_grid_results(gbrt_grid_results_refined)

['learning_rate', 'max_depth', 'n_estimators', 'random_state']
{'default': [0.1, 1, 400, 42], 'standard': [0.1, 1, 400, 42], 'robust': [0.1, 1, 400, 42], 'minmax': [0.1, 1, 400, 42], 'pca': [0.1, 1, 400, 42]}


,learning_rate,max_depth,n_estimators,random_state
default,0.1,1,400,42
standard,0.1,1,400,42
robust,0.1,1,400,42
minmax,0.1,1,400,42
pca,0.1,1,400,42


## KVSM
### linear

In [81]:
ksvm_param_grid = {
    'kernel': ['linear', 'poly', 'rbf'],
    'C': [0.1, 1, 5, 10],
    'degree': [1, 2, 3],
    'gamma': [.1, 1, 10],
    'probability': [True]
}

In [ ]:

ksvm_linear_grid_results_default = grid_search_all_data(SVC(), ksvm_param_grid, data_default, True)

In [82]:

ksvm_linear_grid_results_refined = grid_search_all_data(SVC(), ksvm_param_grid, data_refined, True)

In [ ]:


ksvm_scores_default = score_model_after_cv(
    classifier=SVC(),
    grid_results=ksvm_linear_grid_results_default,
    data=data_default,
    requiresCalibration=False,
    canAcceptNegative=True
)

In [ ]:

pd.DataFrame(ksvm_scores_default)


In [83]:

ksvm_scores_refined = score_model_after_cv(
    classifier=SVC(),
    grid_results=ksvm_linear_grid_results_refined,
    data=data_refined,
    requiresCalibration=False,
    canAcceptNegative=True
)

In [84]:

pd.DataFrame(ksvm_scores_refined)

,default,standard,robust,minmax,pca
train acc,0.970704,0.959437,0.959437,0.982535,0.939718
test acc,0.912162,0.905405,0.864865,0.925676,0.891892
auc,0.992487,0.992305,0.986976,0.995686,0.985836


In [85]:
display_grid_results(ksvm_linear_grid_results_refined)

['C', 'degree', 'gamma', 'kernel', 'probability']
{'default': [0.1, 1, 0.1, 'poly', True], 'standard': [0.1, 1, 0.1, 'linear', True], 'robust': [0.1, 1, 0.1, 'linear', True], 'minmax': [5, 1, 1, 'rbf', True], 'pca': [0.1, 1, 0.1, 'linear', True]}


,C,degree,gamma,kernel,probability
default,0.1,1,0.1,poly,True
standard,0.1,1,0.1,linear,True
robust,0.1,1,0.1,linear,True
minmax,5.0,1,1.0,rbf,True
pca,0.1,1,0.1,linear,True


# MLP


In [12]:
mlp_param_grid = {
    'activation': ['identity', 'relu', 'logistic'],
    'hidden_layer_sizes': [128, 256, 512],
    'alpha': [0.001, 0.005, 0.01]
}

In [13]:

mlp_grid_results_default = grid_search_all_data(MLPClassifier(), mlp_param_grid, data_default, True)


/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maxim

In [66]:

mlp_grid_results_refined = grid_search_all_data(MLPClassifier(), mlp_param_grid, data_refined, True)

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maxim

In [15]:
mlp_scores_default = score_model_after_cv(
    classifier=MLPClassifier(),
    grid_results=mlp_grid_results_default,
    data=data_default,
    requiresCalibration=False,
    canAcceptNegative=True
)

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maxim

In [21]:

pd.DataFrame(mlp_scores_default)

,default,standard,robust,minmax,pca
train acc,0.989296,0.990423,0.986479,0.987042,0.990423
test acc,0.939189,0.930180,0.912162,0.948198,0.932432
auc,0.996150,0.995149,0.993989,0.997203,0.995043


In [67]:

mlp_scores_refined = score_model_after_cv(
    classifier=MLPClassifier(),
    grid_results=mlp_grid_results_refined,
    data=data_refined,
    requiresCalibration=False,
    canAcceptNegative=True
)

/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/engineer/workspace/cse590-machine-learning/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maxim

In [ ]:

pd.DataFrame(mlp_scores_refined)


In [70]:
pd.DataFrame(mlp_scores_refined)

,default,standard,robust,minmax,pca
train acc,0.990423,0.990986,0.987042,0.987606,0.988732
test acc,0.927928,0.925676,0.914414,0.941441,0.898649
auc,0.996224,0.994749,0.993948,0.997166,0.987431


In [71]:
display_grid_results(mlp_grid_results_default)

['activation', 'alpha', 'hidden_layer_sizes']
{'default': ['logistic', 0.001, 128], 'standard': ['relu', 0.005, 128], 'robust': ['logistic', 0.005, 128], 'minmax': ['relu', 0.01, 512], 'pca': ['relu', 0.01, 128]}


,activation,alpha,hidden_layer_sizes
default,logistic,0.001,128
standard,relu,0.005,128
robust,logistic,0.005,128
minmax,relu,0.010,512
pca,relu,0.010,128


In [72]:
display_grid_results(mlp_grid_results_refined)

['activation', 'alpha', 'hidden_layer_sizes']
{'default': ['logistic', 0.01, 256], 'standard': ['relu', 0.01, 256], 'robust': ['logistic', 0.001, 128], 'minmax': ['relu', 0.01, 512], 'pca': ['relu', 0.01, 256]}


,activation,alpha,hidden_layer_sizes
default,logistic,0.010,256
standard,relu,0.010,256
robust,logistic,0.001,128
minmax,relu,0.010,512
pca,relu,0.010,256
